In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import layers

import datetime
import tensorflow_addons as tfa 
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

/opt/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


## Load data

In [2]:
import read_rijksdata
NUM = 10000 # NUMBER OF IMAGES TO USE
MIN_ARTWORK_NUM = 100

In [3]:
img_folder = '/Users/erebor/Downloads/out_img'

In [4]:
X_train, Y_train, label_train, X_val, Y_val, label_val = read_rijksdata.load_data(img_folder=img_folder,
                                                                                  labels_file='labels.txt', 
                                                                                  names_file='names.txt')

In [5]:
names = pd.read_csv('names.txt',delimiter = '/t',header=None,engine='python')
counts = pd.DataFrame(Y_train).value_counts()
min_counts = counts[counts>=MIN_ARTWORK_NUM]
min_idx = [index[0] for index in min_counts.index.tolist()]

min_mask_idx = []
for i, Y_ in enumerate(Y):
    if np.isin(Y_,min_idx):
        min_mask_idx.append(i)

NameError: name 'Y' is not defined

In [ ]:
fig = plt.figure(figsize=(6,5))
fig.patch.set_facecolor('white')

plt.bar(x=list(range(len(counts))),height=list(counts),align='center',width=1)
plt.xlabel('Artist Index')
plt.ylabel('# of Artworks')
plt.title('Artwork Number Distribution')
plt.yscale('log')
plt.axhline(100,color='black')

plt.savefig('artist_cut.png',dpi=300, transparent=False)
plt.show()

In [ ]:
X_ = X_train[min_mask_idx,:,:,:]
Y_ = Y_train[min_mask_idx]

In [ ]:
classes = len(list(set(Y_)))
print('# of classes:',classes)

counts = pd.DataFrame(Y_).value_counts()
print('Min # of artworks for all artists:',min(counts))
print('Min # of artworks specified:',MIN_ARTWORK_NUM)

print('X shape:',X_.shape)
print('Y shape:',Y_.shape)


In [ ]:
classes = len(list(set(Y_)))
Y_hot = tf.one_hot(indices=Y_,depth=classes)
#Y_val = tf.one_hot(indices=Y_val,depth=classes)

## Download and compile model

In [ ]:
input_shape = (56,56,3)
enet_kwargs = {'include_top':False,
               'weights':'imagenet',
               'input_tensor':None,
               'input_shape':input_shape,
               'pooling':None,
               'classes':1,
               'classifier_activation':'softmax'}
enet_base = tf.keras.applications.efficientnet.EfficientNetB0(**enet_kwargs)
#enet_base.trainable = False

In [ ]:
enet = tf.keras.models.Sequential()
enet.add(enet_base)
enet.add(tf.keras.layers.GlobalMaxPooling2D())
enet.add(tf.keras.layers.Dropout(rate=0.01))
enet.add(tf.keras.layers.Dense(classes, activation="softmax"))

In [ ]:
enet.summary()

In [ ]:
# loss function
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,name='categorical_crossentropy')

# metrics

TopKs = []
for k in [1,5]:
    TopK = tf.keras.metrics.TopKCategoricalAccuracy(k=k, name='top_{}'.format(k))
    TopKs.append(TopK)
metrics = ["acc"]
metrics.extend(TopKs)

f1 = tfa.metrics.F1Score(num_classes=classes, threshold=0.05)
metrics.append(f1)

# Optimizer
# very average Adam settings
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile it all
enet.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics)

## Train model

In [ ]:
print(X_.shape, type(X))
print(Y_hot.shape, type(Y_hot))

In [ ]:
history = enet.fit(x=X_,y=Y_hot,validation_split=.20, epochs=5)

In [ ]:
# Save the weights
checkpoint_time = '{date:%Y-%m-%d_%H-%M}'.format(date=datetime.datetime.now())

save_file = './checkpoints/enet_{}'.format(checkpoint_time)
print('Saving to:',save_file)
enet.save_weights(save_file)

In [ ]:
enet.save('models/enet.h5')

## Re-load model and evaluation

In [ ]:
reconstructed_model = keras.models.load_model('models/enet.h5')

In [ ]:
allgood = np.testing.assert_allclose(enet.predict(X_), reconstructed_model.predict(X_))

In [ ]:
print(allgood)

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

confusion_matrix = sklearn.metrics.confusion_matrix(y_true=Y, y_pred=y_pred, labels=None)

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(Y_train, y_pred)
    roc_auc[i] = auc(fpr[i], tpr[i])